In [0]:
from pulp import *

# **Programação Linear em Python: Maximização de lucro utlizando somente  áreas degradadas da Caatinga**



> Este Notebook iPython é um exemplo de construção de um programa linear em Python com o módulo PULP.




---

**Definição do problema:** Você é proprietário de alguns hectares na região da Caatinga e por leis ambientais está limitado utilizar somente regiões degradadas de sua propriedade. Qual é a melhor produção para ser utilizada em cada tipo de terreno e como maximizar o lucro?

---


*   Objetivo: Maximização de lucro utlizando somente áreas degradadas
*   Formula PL: Maximização
*   Variaveis de Decisão: Variavel continua que define o tipo do terreno utilihzado para a produção e quantidade em hectare
*   Restrições: Tamanho das áreas degradadas e produção minima de cada cultura

## Entendendo os dados:



> Nesse exemplo vamos utilizar dois tipos de atividade e dois tipos de região degradada:



In [0]:
tipo_atividade = ["umbu", "caju"]
tipo_regiao = ["mata", "descampado"]
tipo_producao = ["mata_umbu", "mata_caju", "descampado_umbu", "descampado_caju"]



> Primeiro vamos precisar do lucro total em cada regiao **X** relizando a atividade **Y**, para chegar nesse valor poder *subtrair do valor de venda do Kg da atividade pelo o custo de prepararo da região para tal atividade* 



In [0]:
custo_prep_regiao = {
    "mata": 1.5,
    "descampado": 0.4
}

valor_venda_atividade = {
    "umbu": 5,
    "caju": 3
}

qtd_lucro_regiao = {
    "mata_umbu": valor_venda_atividade["umbu"] - custo_prep_regiao["mata"],
    "mata_caju": valor_venda_atividade["caju"] - custo_prep_regiao["mata"],
    "descampado_umbu": valor_venda_atividade["umbu"] - custo_prep_regiao["descampado"],
    "descampado_caju": valor_venda_atividade["caju"] - custo_prep_regiao["descampado"]
}



> Proximo passo é definir a quantidade de terra que o proprietario possui de cara região, o quanto ele consegue produzir de cada atividade em cada uma dessas regiões



In [0]:
qtd_hectare_regiao = {
    "mata": 5,
    "descampado": 3,
}

qtd_prod_regiao = {
    "mata_umbu": 420,
    "mata_caju": 500,
    "descampado_umbu": 350,
    "descampado_caju": 470
}



> Por ultimo vamos definir restrições contratuais das atividade: esses dados são utilizados para garantir que o proprietario produza a quantidade minima de cada atividade. 



In [0]:
qtd_contrato_atividade = {
    "umbu": 1430,
    "caju": 1500
}

## Definição do Probblema



> Vamos começar definindo um problema de maximização utilizando o PULP



In [0]:
problem = LpProblem('ProjetoSertao', LpMaximize)



> Definindo as variaveis de decisão do problema, onde vamos ter o quanto de cada região vamos utilizar para um tipo de atividade



In [0]:
atividades_vars = LpVariable.dicts("atividades", tipo_producao, lowBound=0)




---

> Definindo a função objetivo do problema, onde buscamos maximinizar o lucro do proprietario da região. Utilizando a equação:

\begin{equation*}
\sum_{i=1}^k A_k L_k P_k
\end{equation*}

*Onde:*

*A* é a quantidade de hectares necessaria para a atividade *k* por região;

*L* é o lucro da atividade *k* por unidade produzida por região;

*P* é a producao de cada região por atividade *k*;



---




In [0]:
problem += lpSum([atividades_vars[tp] * qtd_lucro_regiao[tp] * qtd_prod_regiao[tp] for tp in tipo_producao]), "Funcao objetivo"



---



> Definido a função objetivo podemos partir para as funções de restrições do tamanho das regiões, sendo:

\begin{equation*}
\sum_{i=1}^h A_h \leq H
\end{equation*}

*Onde:*

*A* é a atividade que vai ser realizada em uma região *h*;

*H* é a quantidade total de hectare da região *h*



---




In [0]:
problem += lpSum([atividades_vars[tp] for tp in tipo_producao if "mata" in tp]) <= qtd_hectare_regiao["mata"]
problem += lpSum([atividades_vars[tp] for tp in tipo_producao if "descampado" in tp]) <= qtd_hectare_regiao["descampado"]



---



> E as funções de restrição contratuais, sendo:


\begin{equation*}
\sum_{i=1}^c P_c A_c \geq C
\end{equation*}

*Onde:*

*P* é a produção da atividade *c* nas regiões da propriedade;

*A* é a atividade *c* que vai ser realizada em uma das regiões da propriedade;

*C* é a quantidade minima a ser atingida de produção de uma atividade *c*;



---



In [0]:
problem += lpSum([qtd_prod_regiao[tp] * atividades_vars[tp] for tp in tipo_producao if "umbu" in tp]) >= qtd_contrato_atividade["umbu"]
problem += lpSum([qtd_prod_regiao[tp] * atividades_vars[tp] for tp in tipo_producao if "caju" in tp]) >= qtd_contrato_atividade["caju"]



> Finalmente, a definição do modelo ficou assim:



In [12]:
problem

ProjetoSertao:
MAXIMIZE
1222.0*atividades_descampado_caju + 1609.9999999999998*atividades_descampado_umbu + 750.0*atividades_mata_caju + 1470.0*atividades_mata_umbu + 0.0
SUBJECT TO
_C1: atividades_mata_caju + atividades_mata_umbu <= 5

_C2: atividades_descampado_caju + atividades_descampado_umbu <= 3

_C3: 350 atividades_descampado_umbu + 420 atividades_mata_umbu >= 1430

_C4: 470 atividades_descampado_caju + 500 atividades_mata_caju >= 1500

VARIABLES
atividades_descampado_caju Continuous
atividades_descampado_umbu Continuous
atividades_mata_caju Continuous
atividades_mata_umbu Continuous

## Resultados:



> Após toda a definição do problema, com suas restrições e variavies de decisão. Podemos chamar a função abaixo para ver a solução gerada pelo solver:




In [13]:
problem.solve()

1



> O solver aprensenta algumas definições do resultado apresentado pela otimização, podendo ser:

1.   Ótima
2.   Não resolvido
3.   Inviável
4.   Não vinculado
5.   Indefinido

> Nesse exemplo, ele encontrou a seguinte solução:

In [14]:
print("Solução:", LpStatus[problem.status])

Solução: Optimal




> O modelo apresentou a seguinte distribuição de atividade por tipo do terreno, os valores apresentados em cada variavel abaixo é a quantidade de hectares que deve ser utilizado por cada produção.



In [15]:
for v in problem.variables():
    print(f'{v.name}:', f'{v.varValue}ha')

atividades_descampado_caju: 3.0ha
atividades_descampado_umbu: 0.0ha
atividades_mata_caju: 0.18ha
atividades_mata_umbu: 4.82ha




> Resultado final da equação de otimização:



In [16]:
print('R$ {:.2f}'.format(value(problem.objective)))

R$ 10886.40
